In [ ]:
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import os
import cv2 as cv
import pydicom as pyd
from histogram_equalization import preprocess,watersheding,validation,alpha_seg

ddd = './Data/dicom_dir/'               #Data Dir DICOM
#Wddt = "./Data/tiff_images/"             #Data Dir Tiff

In [ ]:
im = pyd.dcmread(ddd+os.listdir(ddd)[2])
plt.figure(figsize=(10,10))
plt.imshow(im.pixel_array,cmap='gray')

In [ ]:
plt.figure(figsize=(10,10))
plt.imshow(preprocess(im,'mg','conv')[0],cmap='gray')

In [ ]:
# Define the images that will be used for the analysis process
img1 = preprocess(pyd.dcmread(ddd+os.listdir(ddd)[1]),'mg','conv')[0]
img2 = preprocess(pyd.dcmread(ddd+os.listdir(ddd)[0]),'mg','conv')[0]
img3 = preprocess(pyd.dcmread(ddd+os.listdir(ddd)[2]),'mg','conv')[0]

fig, ax = plt.subplots(2,3,figsize=(25,10),gridspec_kw={'height_ratios': [5, 1]})
ax[0,0].imshow(img1,cmap='gray')
ax[0,1].imshow(img2,cmap='gray')
ax[0,2].imshow(img3,cmap='gray')

ax[1,0].hist(img1.flatten(),bins=255)
ax[1,1].hist(img2.flatten(),bins=255)
ax[1,2].hist(img3.flatten(),bins=255)
plt.show()

# Otsu Segmentation for seeding the images

In [ ]:
def f_b_seg(img,t1,t2):
    img_map = []
    for x in img.flatten():
        if  t1<= x <=t2:
            img_map.append(1)
        else:
            img_map.append(0)
    img_map = np.asarray(img_map).reshape(img.shape)

    return img_map

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(25,10))
ax[0].imshow(f_b_seg(img1,130,150),cmap='gray')
ax[1].imshow(f_b_seg(img2,185,250),cmap='gray')
ax[2].imshow(f_b_seg(img3,130,140),cmap='gray')
plt.show()

# Segmented Images
img1_seg = f_b_seg(img1,130,150)
img2_seg = f_b_seg(img2,170,200)
img3_seg = f_b_seg(img3,130,140)

# Active Contours

In [ ]:
def store_evolution_in(lst):
    """Returns a callback function to store the evolution of the level sets in
    the given list.
    """

    def _store(x):
        lst.append(np.copy(x))

    return _store

In [ ]:
from skimage.segmentation import morphological_geodesic_active_contour,inverse_gaussian_gradient,disk_level_set

fig, ax = plt.subplots(1,3,figsize=(25,10))

#Plot Images
ax[0].imshow(img1,cmap='gray')
ax[1].imshow(img2,cmap='gray')
ax[2].imshow(img3,cmap='gray')


# Initial level set
init_ls = np.zeros(img1.shape, dtype=np.int8)
init_ls[300:400, 300:410] = 1
evolution = []
callback = store_evolution_in(evolution)
# List with intermediate results for plotting the evolution
ls1 = morphological_geodesic_active_contour(inverse_gaussian_gradient(img1),iterations=230,
                                           init_level_set=init_ls,
                                           smoothing=1, balloon=1,
                                           threshold=0.69,iter_callback=callback)
ax[0].contour(ls1)
#ax[0].set_title(cv.contourArea(ls1.astype('uint8')))

# Initial level set
"""init_ls = np.zeros(img2.shape, dtype=np.int8)
init_ls[350:400, 350:400] = 1"""
init_ls = disk_level_set(img2.shape,center=[380,380],radius=30)
# List with intermediate results for plotting the evolution
ls2 = morphological_geodesic_active_contour(inverse_gaussian_gradient(img2),iterations=230,
                                           init_level_set=init_ls,
                                           smoothing=1, balloon=-2,
                                           threshold=0.4,iter_callback=callback)
ax[1].contour(ls2)
#ax[1].set_title(cv.contourArea(ls2.astype('uint8')))

# Initial level set
init_ls = np.zeros(img3.shape, dtype=np.int8)
init_ls[240:300, 150:200] = 1
# List with intermediate results for plotting the evolution
ls3 = morphological_geodesic_active_contour(inverse_gaussian_gradient(img3),iterations=230,
                                           init_level_set=init_ls,
                                           smoothing=1, balloon=-1,
                                           threshold=0.69,iter_callback=callback)
ax[2].contour(ls3)
#[2].set_title(cv.contourArea(ls3.astype('uint8')))


# WaterShed

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(25,10))

ax[0].imshow(img1,cmap='gray')
area,c1,l = watersheding(img1,img1_seg)
ax[0].contour(c1[76])
ax[0].set_title(f'Area = {area[76]}')

ax[1].imshow(img2,cmap='gray')
area,c2,l = watersheding(img2,img2_seg)
ax[1].contour(c2[35])
ax[1].set_title(f'Area = {area[35]}')

ax[2].imshow(img3,cmap='gray')
area,c3,l = watersheding(img3,img3_seg)
ax[2].contour(c3[20])
ax[2].set_title(f'Area = {area[20]}')

# Validation

In [ ]:
# True annotations
anno1,anno2,anno3 = plt.imread('./Data/Annotations/annot2.jpg')[:,:,0], plt.imread('./Data/Annotations/annot1.jpg')[:,:,0], plt.imread('./Data/Annotations/annot3.jpg')[:,:,0]
# Morphological snake
morph1,morph2,morph3 = ls1,ls2,ls3
#Watershed
water1,water2,water3 = c1[76],c2[35],c3[20]

In [ ]:
validation(anno1,morph1,water1)

In [ ]:
validation(anno2,morph2,water2)

In [ ]:
validation(anno3,morph3,water3)

# Alpha Beta and Alpha expansion

In [ ]:
s1,e1 = alpha_seg(img1,img1_seg)

In [ ]:
s2,e2 = alpha_seg(img2,img2_seg)

In [ ]:
s3,e3 = alpha_seg(img3,img3_seg)

In [ ]:
mask = np.zeros(s1.shape[:2], dtype="uint8")
cv.rectangle(mask, (300, 300), (400, 400), 255, -1)
s1 = cv.bitwise_and(s1, s1, mask=mask)
e1 = cv.bitwise_and(e1, e1, mask=mask)


mask = np.zeros(s2.shape[:2], dtype="uint8")
cv.circle(mask, (380,380), 30, 255, -1)
s2 = cv.bitwise_and(s2, s2, mask=mask)
e2 = cv.bitwise_and(e2, e2, mask=mask)


mask = np.zeros(s3.shape[:2], dtype="uint8")
cv.rectangle(mask, (150, 300), (200, 240), 255, -1)
s3 = cv.bitwise_and(s3, s3, mask=mask)
e3 = cv.bitwise_and(e3, e3, mask=mask)

In [ ]:
fig, ax = plt.subplots(2,3,figsize=(15,10))
ax[0,0].imshow(s1,cmap='gray')
ax[0,1].imshow(s2,cmap='gray')
ax[0,2].imshow(s3,cmap='gray')
ax[1,0].imshow(e1,cmap='gray')
ax[1,1].imshow(e2,cmap='gray')
ax[1,2].imshow(e3,cmap='gray')
plt.show()

In [ ]:
validation(anno1,s1,e1,idx=['Alpha Beta Swap','Alpha Expansion'])

In [ ]:
validation(anno2,s2,e2,idx=['Alpha Beta Swap','Alpha Expansion'])

In [ ]:
validation(anno3,s3,e3,idx=['Alpha Beta Swap','Alpha Expansion'])